<a href="https://colab.research.google.com/github/pointmina/practicalNLP/blob/main/model_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [2]:
from datasets import load_dataset   # Huggingface 데이터셋 패키지 import
data = load_dataset("sepidmnorozy/Korean_sentiment")    # 데이터 다운로드

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/sepidmnorozy___csv/sepidmnorozy--Korean_sentiment-c4fbcaaf8f2ffcfa/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
from sklearn.feature_extraction.text import CountVectorizer       # CountVectorizer: Bag of words 벡터화 구현을 하기 위한 클래스

In [ ]:
# connect google drive
from google.colab import drive
drive.mount('/content/drive')
# Download konlpy
%cd ./drive/MyDrive/Colab\ Notebooks/
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd ./Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab_light_220429.sh

In [9]:
from konlpy.tag import Mecab
mecab = Mecab()

In [10]:
train_data = data['train']
dev_data = data['validation']
test_data = data['test']

#형태소 단위로 tokenization 된 텍스트를 {train/dev/test}_data에 저장
train_data = train_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
dev_data = dev_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})
test_data = test_data.map(lambda example: {'label': example['label'], 'text': ' '.join(mecab.morphs(example['text']))})

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1333 [00:00<?, ? examples/s]

Map:   0%|          | 0/2667 [00:00<?, ? examples/s]

In [21]:
#CountVectorizer 수정, N-gram(1,2), 형태소 분석 입력

vectorizer = CountVectorizer(strip_accents='unicode', token_pattern=r"(?u)\b\w\w+\b|'\w+", ngram_range=(1,2))
vectorizer.fit(train_data['text'])

#print("Vectorizer vocab: ", vectorizer.vocabulary_)
print("Vectorizer vocab size: ", len(vectorizer.vocabulary_))

train_vectors = vectorizer.transform(train_data['text'])
dev_vectors = vectorizer.transform(dev_data['text'])
test_vectors = vectorizer.transform(test_data['text'])

# print(train_vectors)

Vectorizer vocab size:  223954


In [22]:
sample_num = 0
sample_origin = train_data[sample_num]
sample_transform = train_vectors[sample_num]
sample_inverse_transform = vectorizer.inverse_transform(sample_transform)
print("Original Input:{}\nTransformed: {}\nInv-transformed: {}".format(sample_origin, sample_transform, sample_inverse_transform))

Original Input:{'label': 1, 'text': '역시 명작 어렸 을 때 봤 을 때 도 재밌 었 고 지금 봐도 몇 억 배 이상 으로 재밌 어요'}
Transformed:   (0, 15316)	1
  (0, 17657)	1
  (0, 60074)	1
  (0, 61435)	1
  (0, 68655)	2
  (0, 68745)	1
  (0, 68834)	1
  (0, 85608)	1
  (0, 85662)	1
  (0, 85794)	1
  (0, 85829)	1
  (0, 94206)	1
  (0, 94251)	1
  (0, 99425)	1
  (0, 99494)	1
  (0, 100109)	1
  (0, 100188)	1
  (0, 130302)	1
  (0, 130304)	1
  (0, 131104)	1
  (0, 132029)	1
  (0, 132038)	1
  (0, 133487)	1
  (0, 133493)	1
  (0, 139609)	1
  (0, 139721)	1
  (0, 152851)	1
  (0, 153607)	1
  (0, 156943)	2
  (0, 157495)	2
  (0, 168985)	1
  (0, 169051)	1
  (0, 178035)	2
  (0, 178100)	1
  (0, 178105)	1
  (0, 189961)	1
  (0, 190023)	1
Inv-transformed: [array(['고', '고 지금', '도', '도 재밌', '때', '때 도', '때 봤',
       '명작', '명작 어렸', '몇', '몇 억', '배', '배 이상',
       '봐도', '봐도 몇', '봤', '봤 을', '어렸', '어렸 을', '어요',
       '억', '억 배', '었', '었 고', '역시', '역시 명작', '으로',
       '으로 재

In [23]:
##### 훈련세트에서 훈련하고 평가하기
from sklearn.svm import LinearSVC     #Linear kernel SVM을 사용하기 위한 클래스     

svm = LinearSVC()     # 모델 정의
svm.fit(train_vectors, train_data['label'])     # 모델 학습

print(svm.coef_)      # weights (w)
print(svm.intercept_)     # bias (b)

[[-0.11567922 -0.11567922  0.         ...  0.04532829  0.
   0.        ]]
[-0.1118595]


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [24]:
from sklearn.metrics import accuracy_score    # Accuracy 측정 함수 import

final_model = svm
pred_results = final_model.predict(test_vectors)    # 최종 모델로 test 데이터 예측
accuracy = accuracy_score(test_data['label'], pred_results)   # 정확도 측정

print("Accuracy: {:.2f}%".format(accuracy*100))   # 정확도 출력 

Accuracy: 84.03%
